In [1]:
from openai import AsyncOpenAI
import asyncio
import re
import sqlite3
# OpenAI API 키 설정
client = AsyncOpenAI()

In [2]:
async def text_to_sql_query(question):
    # LLM에 전달할 프롬프트 생성
    instruction = """
    You are an expert database administrator.
    Given a natural language question and the table schema, generate an SQL query that answers the question.

    Table Schema:
    CREATE TABLE campaign_board (
        campaign_id INTEGER PRIMARY KEY AUTOINCREMENT,        -- 캠페인 고유 ID
        title TEXT NOT NULL,                                  -- 캠페인 제목
        description TEXT,                                     -- 캠페인 설명
        created_by TEXT NOT NULL,                             -- 작성자 (캠페인 등록자)
        start_date DATE,                                      -- 캠페인 시작일
        end_date DATE,                                        -- 캠페인 종료일
        total_funds REAL DEFAULT 0,                           -- 모금된 총 금액
        view_count INTEGER DEFAULT 0,                         -- 게시글 조회수
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,       -- 게시글 생성일
        updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP        -- 게시글 수정일
    );

    Generate an appropriate SQL query based on the question.

    Question: """ + question + """

    SQL Query:
"""

    # OpenAI GPT-4 모델 호출
    response = await client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": instruction},  # 시스템 인스트럭션으로 제공
            {"role": "user", "content": question}          # 유저의 실제 프롬프트 제공
        ],
        max_tokens=150,
        temperature=0,
    )
    
    # 생성된 SQL 쿼리 추출
    sql_query = response.choices[0].message.content

    return sql_query

async def clean_sql_query(sql_query):
    sql_query = re.sub(r'^```(?:sql)?', '', sql_query, flags=re.IGNORECASE).strip()
    sql_query = re.sub(r'```$', '', sql_query).strip()
    return sql_query
    
async def execute_query(sql_query):
    conn = sqlite3.connect('test_database.db')
    cursor = conn.cursor()

    try:
        cursor.execute(sql_query)
        rows = cursor.fetchall()

        return rows

    except sqlite3.Error as e:
        return f"An error occurred: {e.args[0]}"

    finally:
        conn.close()

async def process_question(question):
    # SQL 쿼리 생성
    sql_query = await text_to_sql_query(question)
    clean_query = await clean_sql_query(sql_query)

    # SQL 쿼리 실행
    result = await execute_query(clean_query)

    # 반환된 SQL 쿼리, 정리된 쿼리, 그리고 결과를 함께 반환
    return clean_query, result


In [3]:
# 예시 질문 텍스트
question_texts = [
        "행사 모금이 가장 많이된 상위 5개의 제목을 알려줄래?",
        "조회수가 가장 높은 캠페인들은 무엇인가요?",
        "이번 달에 시작한 캠페인 목록을 보여주세요.",
        "admin이 만든 캠페인 중 모금액이 100000 이상인 것은?",
        "캠페인 종료일이 가까운 순서대로 3개를 알려주세요.",
        "모금액이 50000 미만인 캠페인은 무엇인가요?",
        "캠페인 제목에 '환경'이 포함된 것들을 보여주세요.",
        "오늘 생성된 캠페인들은 무엇이 있나요?",
        "view_count가 100 이상인 캠페인들의 제목과 조회수를 알려주세요.",
        "모금액 대비 조회수가 높은 캠페인을 알려줄 수 있나요?",
        "행사 모금이 가장 많이된 상위 5개의 제목을 알려줄래?",
    ]
    

In [4]:
async def main():
    # 비동기적으로 모든 질문 처리
    results = await asyncio.gather(*(process_question(question) for question in question_texts))

    # 각 질문에 대해 출력
    for question, (clean_query, result) in zip(question_texts, results):
        print(f"Q: {question}")
        print(f"Query: {clean_query}")
        print(f"A: {result}\n")

In [5]:
# In Jupyter or other environments where event loop is already running
if __name__ == "__main__":
    try:
        # This will work when no loop is running
        asyncio.run(main())
    except RuntimeError:
        # If we're already in an event loop, use this:
        await main()

Q: 행사 모금이 가장 많이된 상위 5개의 제목을 알려줄래?
Query: SELECT title
FROM campaign_board
ORDER BY total_funds DESC
LIMIT 5;
A: [('환경 보호 캠페인 11',), ('환경 보호 캠페인 25',), ('환경 보호 캠페인 24',), ('환경 보호 캠페인 23',), ('환경 보호 캠페인 22',)]

Q: 조회수가 가장 높은 캠페인들은 무엇인가요?
Query: SELECT * 
FROM campaign_board 
ORDER BY view_count DESC 
LIMIT 10;  -- 상위 10개의 캠페인 조회수 기준
A: [(25, '환경 보호 캠페인 25', '지구를 위한 환경 보호 활동 25', 'user7', '2024-01-01', '2024-12-31', 180000.0, 200, '2024-10-21 04:47:14', '2024-10-21 04:47:14'), (24, '환경 보호 캠페인 24', '지구를 위한 환경 보호 활동 24', 'user6', '2024-12-01', '2024-12-31', 170000.0, 190, '2024-10-21 04:47:14', '2024-10-21 04:47:14'), (22, '환경 보호 캠페인 22', '지구를 위한 환경 보호 활동 22', 'user4', '2024-10-01', '2024-12-31', 155000.0, 170, '2024-10-21 04:47:14', '2024-10-21 04:47:14'), (11, '환경 보호 캠페인 11', '지구를 위한 환경 보호 활동 11', 'user9', '2024-11-01', '2024-12-31', 200000.0, 150, '2024-10-21 04:47:14', '2024-10-21 04:47:14'), (23, '환경 보호 캠페인 23', '지구를 위한 환경 보호 활동 23', 'user5', '2024-11-01', '2024-12-31', 160000.0, 150, 

/tmp/ipykernel_3905196/658487833.py:8: RuntimeWarning: coroutine 'main' was never awaited
  await main()
